In [16]:
import pandas as pd 
import wandb
from tqdm import tqdm
import requests
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.colors as mcolors

In [17]:
BASE_METRIC = "accuracy_per_mean_user_and_bot"
WANDB_OR_USERNAME = "guylororg"


In [69]:
def result_metric(sweeps, group_name, drop_list=[], drop_HPT=False, metric=BASE_METRIC, epoch="best"):
    df = api.get_sweeps_results(sweeps, metric=metric) 

    config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
    HPT_cols = [col for col in config_cols if df[col].nunique() > 1]
    # print(HPT_cols)
    if drop_HPT:
        df=df.drop([c for c in HPT_cols if not c in ["config_LLM_SIM_SIZE", "config_seed"]], axis=1)
        HPT_cols = ["config_LLM_SIM_SIZE", "config_seed"]
    # display(df['config_online_simulation_factor'])
    numeric_cols = df.select_dtypes(include='number').columns.tolist()
    grouped = df.groupby([c for c in HPT_cols if c != "config_seed"])
    
    mean_df = grouped[numeric_cols].mean()
    std_df = grouped[numeric_cols].std()
    # display(mean_df)

    if epoch=="best":
        best_col = mean_df[[c for c in mean_df.columns if (metric in c and metric[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)
    else:
        best_col = mean_df[[c for c in mean_df.columns if f"{metric}_epoch{epoch}" in c]].idxmax(axis=1)
    
    result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
    means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
    stds = grouped.apply(lambda x: x[best_col.loc[x.name]].std())

    # display(result)


    df_cols = {'mean': means, 'std': stds, 'values': result.values}
    # display(df_cols)
    if epoch == "best": df_cols['epoch'] = best_col.apply(lambda x: int(x.split("epoch")[1]) if "epoch" in x else "last")

    df_cols['CI'] = result.apply(lambda x: bootstrap_ci(x))
    # display(df_cols)
    summary_df = pd.DataFrame(df_cols, index=best_col.index)
    # display(summary_df)
    for d in drop_list:
        if d in summary_df.index:
            summary_df=summary_df.drop(d)
    # if len(summary_df.index.names) == 1:
    #     return summary_df.rename_axis(group_name)
    # else:
    #     return summary_df

    return summary_df

def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
    bootstrapped_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_means.append(np.mean(sample))
    lower_bound = np.percentile(bootstrapped_means, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_means, (1 + ci) / 2 * 100)
    return lower_bound, upper_bound


# For HyperParameterTuning

For every configuration that you test in the sweep, you will receive in the table the average, standard deviation, all the values obtained for the different seed values, and also the confidence interval within which the result is located at a confidence level of 95%.

When epoch="best" is defined, you can check in which epoch the best result is obtained. If epoch=5 is defined, you will receive the result obtained for epoch number 5.

You can test multiple sweeps simultaneously by entering them into the list found in the first element of the function result_metric.

In [66]:
from read_wandb import wandb_results
# api = wandb_results("NLP2024_PROJECT_testing_action_1", wandb_username=WANDB_OR_USERNAME)
# api = wandb_results("NLP2024_PROJECT_wandb_check", wandb_username=WANDB_OR_USERNAME)
api = wandb_results("NLP2024_PROJECT_testing_action_parallel", wandb_username=WANDB_OR_USERNAME)
# sweep_ids_list = ['decuirq2']
sweep_ids_list = ["yiteamf1"]

In [67]:
df = api.get_sweeps_results(sweep_ids_list, metric=BASE_METRIC)
config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
HPT_cols = [col for col in config_cols if df[col].nunique() > 1]
print(HPT_cols)

Total number of sweeps: 1
Download sweep_id='yiteamf1' data...


['config_seed', 'config_online_simulation_factor']


In [68]:
sweep_results = result_metric(sweep_ids_list, "config_online_simulation_factor",drop_list=[], drop_HPT=False, epoch="best")
sweep_results

Total number of sweeps: 1
Download sweep_id='yiteamf1' data...


,mean,std,values,epoch,CI
config_online_simulation_factor,,,,,
0,0.828898,0.001633,"[0.8267503772423906, 0.8277120396253037, 0.829...",8,"(0.8276611248201542, 0.8301356815345793)"
2,0.833129,0.002834,"[0.8318467475847794, 0.8298785538482785, 0.834...",15,"(0.8311417926847199, 0.8355283712881759)"


In [38]:
# sweep_results = result_metric(["decuirq2"], "LLMs", drop_HPT=False, epoch="best")
# sweep_results

# Result for a specific epoch

In [10]:
sweep_results = result_metric(sweep_ids_list, "config_online_simulation_factor", drop_HPT=False, epoch=20)
sweep_results

Total number of sweeps: 1
Download sweep_id='yiteamf1' data...
['config_seed', 'config_online_simulation_factor']


,mean,std,values,CI
config_online_simulation_factor,,,,
2,0.831352,0.003597,"[0.8294915016670176, 0.8258698902080929, 0.833...","(0.8281176298981829, 0.833818391352394)"
